In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  6 13:30:49 2021

rvu -vp -7 0 3 -vd 1 0 0 Sim1.oct
rvu -vp 0 -5 3 -vd 0 1 0 Sim1.oct

@author: sayala
"""

In [1]:
import bifacial_radiance
import pytest

import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'makeModTests')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sarefeen\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\makeModTests


In [2]:
demo = bifacial_radiance.RadianceObj('FrameOmegaTest', testfolder)

path = C:\Users\sarefeen\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\makeModTests


In [3]:
#generating sky

x = 2
y = 1
xgap = 0.02
ygap = 0.15
zgap = 0.10
numpanels = 1
offsetfromaxis = True
Ny = numpanels  

module_type = 'TEST'
frameParams = {'frame_material' : 'Metal_Grey', 
               'frame_thickness' : 0.003,
               'frame_z' : 0.03,
               'nSides_frame' : 4,
               'frame_width' : 0.05}


omegaParams = {'omega_material': 'litesoil',
                'x_omega1' : 0.10,
                'mod_overlap' : 0.5,
                'y_omega' : 1.5,
                'x_omega3' : 0.05,
                'omega_thickness' : 0.01,
                'inverted' : False}

nMods = 1
nRows = 1
sceneDict = {'tilt':0, 'pitch':3, 'clearance_height':3,'azimuth':90, 'nMods': nMods, 'nRows': nRows} 

In [4]:
demo.setGround(0.2)
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)
metdata = demo.readWeatherFile(epwfile, coerce_year = 2021)
demo.gendaylit(4020)

Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!
start: 2021-01-01 01:00:00
end: 2021-12-31 23:00:00
Saving file EPWs\epw_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


C:\Users\sarefeen\Anaconda3\lib\site-packages\pvlib\solarposition.py:448: FutureWarning: casting datetime64[ns, UTC] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  unixtime = np.array(utcday.astype(np.int64)/10**9)
C:\Users\sarefeen\Anaconda3\lib\site-packages\pvlib\solarposition.py:368: FutureWarning: casting datetime64[ns, pytz.FixedOffset(-300)] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  unixtime = np.array(time.astype(np.int64)/10**9)


'skies\\sky2_37.5_-77.33_2021_06_17_13_00.rad'

In [5]:
loopaxisofRotation = [True, True, True, True, True, True, True, True]
loopTorquetube = [True, True, True, True, False, False, False, False ]
loopOmega = [omegaParams, omegaParams, None, None, omegaParams, omegaParams, None, None]
loopFrame = [frameParams, None, frameParams, None, frameParams,  None, frameParams, None]
expectedModuleZ = [3.179, 3.149, 3.179, 3.149, 3.129, 3.099, 3.129, 3.099]

In [7]:
assertionResults = []
for ii in range (0, len(loopOmega)):
    omegaParamsint = loopOmega[ii]
    frameParamsint = loopFrame[ii]
    axisofrotationTorqueTube = loopaxisofRotation[ii]
    torquetube = loopTorquetube[ii]
    
    diam = 0.1
    if torquetube is False:
        diam = 0.0
        
    demo.makeModule(name=module_type,x=x, y=y, torquetube = True, 
                    diameter = diam, xgap = xgap, ygap = ygap, zgap = zgap, 
                    numpanels = Ny, frameParams=frameParams, omegaParams=omegaParams,
                    axisofrotationTorqueTube=axisofrotationTorqueTube)
    
    scene = demo.makeScene(module_type,sceneDict)
    octfile = demo.makeOct()
    analysis = bifacial_radiance.AnalysisObj()  # return an analysis object including the scan dimensions for back irradiance
    frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=1) # Gives us the dictionaries with coordinates
    assertionResults.append(backscan['zstart'])
    # assert backscan['zstart'] == expectedModuleZ[ii]
    
#assert assertionResults == expectedModuleZ


Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Postion 1 Offsetfromaxis: 0.18
NEW CHANGE HERE!!!!
Module TEST updated in module.json
Created FrameOmegaTest.oct

Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Postion 1 Offsetfromaxis: 0.18
NEW CHANGE HERE!!!!
Module TEST updated in module.json
Created FrameOmegaTest.oct

Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Postion 1 Offsetfromaxis: 0.18
NEW CHANGE HERE!!!!
Module TEST updated in module.json
Created FrameOmegaTest.oct

Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Postion 1 Offsetfromaxis: 0.18
NEW CHANGE HERE!!!!
Module TEST updated in module.json
Created FrameOmegaTest.oct

Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Postion 1 Offsetfromaxis: 0.13
NEW CHANGE HERE!!!!
Module TEST updated in module.json
Created FrameOmegaTest.oct

Module Name: TEST
Pre-existing .rad fil

In [8]:
assertionResults == expectedModuleZ

False

In [9]:
for i in range(len(assertionResults)):
    if assertionResults[i] != expectedModuleZ[i]:
        print('Omega =',loopOmega[i] is not None, end = ', ')
        print('Frame =',loopFrame[i] is not None, end = ', ')
        print('Axis of Rotation =',loopaxisofRotation[i], end = ', ')
        print('Torquetube =',loopTorquetube[i], end = ', ')
        print('Off by =',assertionResults[i] - expectedModuleZ[i], end = '\n')
    

Omega = True, Frame = False, Axis of Rotation = True, Torquetube = True, Off by = 0.029999999999999805
Omega = False, Frame = False, Axis of Rotation = True, Torquetube = True, Off by = 0.029999999999999805
Omega = True, Frame = False, Axis of Rotation = True, Torquetube = False, Off by = 0.029999999999999805
Omega = False, Frame = False, Axis of Rotation = True, Torquetube = False, Off by = 0.029999999999999805
